In [3]:
import pygame
import time

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
drum_path = "E:/Documents/University/2018 Semester 2/COMP3419 Graphics and Multimedia/Labs/Lab Materials/drum.wav"
sound_path = "E:/Documents/University/2018 Semester 2/COMP3419 Graphics and Multimedia/Labs/Lab Materials/sample-section1-2"

## Task 1 Mouse Clicking to Play Sound

In [4]:
pygame.init() #initialising pygame
pygame.mixer.init()
screen = pygame.display.set_mode((400,300))
pygame.mixer.music.load(drum_path)

done = False
while not done: 
    for event in pygame.event.get(): #get all events from pygame 
        if event.type == pygame.QUIT:
            done = True 
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 2: #middle mouse button 
                print('play')
                pygame.mixer.music.play(0)
                time.sleep(1) 
            elif event.button == 1: 
                print('pause') #left buttom to pause 
                pygame.mixer.music.pause() 
            else: #right button to play again
                print('play again') 
                pygame.mixer.music.unpause() 
    pygame.display.flip() #This will update the contents of the entire display
pygame.quit()

## Task 2 Control the volume with mouse position

In [6]:
pygame.init() #initialising pygame
pygame.mixer.init()
screen = pygame.display.set_mode((400,300))
pygame.mixer.music.load(drum_path)

done = False
while not done: 
    for event in pygame.event.get(): #get all events from pygame 
        if event.type == pygame.QUIT:
            done = True 
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 2: #middle mouse button 
                print('play')
                pygame.mixer.music.play(0)
                time.sleep(1) 
            elif event.button == 1: 
                print('pause') #left buttom to change position and volume 
                pygame.mixer.music.pause() 
                mx, my = pygame.mouse.get_pos() #get current position of the mouse 
                print('position (x,y):', mx, my)
                rate = my/300.0
                pygame.mixer.music.set_volume(rate) #change volume. Rate is a float in range 
                #(0,1). The new voume is previous_volume * rate so can't make volume louder 
            else: #right button to play again
                print('play again') 
                pygame.mixer.music.unpause() 
    pygame.display.flip() #This will update the contents of the entire display
pygame.quit()

## Task 3 Processing - sound filtering 

In [7]:
from pydub import AudioSegment
from pydub.playback import play 
from pydub.scipy_effects import low_pass_filter, high_pass_filter

D:\Anaconda\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
D:\Anaconda\lib\site-packages\pydub\utils.py:179: RuntimeWarning: Couldn't find ffplay or avplay - defaulting to ffplay, but may not work
  warn("Couldn't find ffplay or avplay - defaulting to ffplay, but may not work", RuntimeWarning)


In [8]:
#AudioSegment.ffmpeg = "D:/ffmpeg/bin"
song = AudioSegment.from_wav(sound_path)
play(song)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Documents/University/2018 Semester 2/COMP3419 Graphics and Multimedia/Labs/Lab Materials/sample-section1-2'

In [24]:
low_pass = low_pass_filter(song, 300) #keep the sounds with frequency lower than 300Hz
play(low_pass)

In [23]:
high_pass = high_pass_filter(song, 300)
play(high_pass)